In [7]:
%%sh
mkdir efs

In [ ]:
%%sh
sudo yum install amazon-efs-utils

## Notebook per descarregar dades

In [2]:
from datetime import datetime as dt
from datetime import timedelta
import argparse
import gzip
import glob
import os
import shutil
import time
import boto3

import requests


### Montar EFS

In [3]:
conn = boto3.client('efs')
conn.describe_file_systems()


{'ResponseMetadata': {'RequestId': 'aa868fd0-5852-4abe-bb4d-9e1fb11e4753',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aa868fd0-5852-4abe-bb4d-9e1fb11e4753',
   'content-type': 'application/json',
   'content-length': '804',
   'date': 'Wed, 29 Jun 2022 21:27:28 GMT'},
  'RetryAttempts': 0},
 'FileSystems': [{'OwnerId': '050980610391',
   'CreationToken': 'console-6cc3914d-4c75-4c84-9c1e-ae2809802dcf',
   'FileSystemId': 'fs-019a6b4db8268f5f0',
   'FileSystemArn': 'arn:aws:elasticfilesystem:eu-west-2:050980610391:file-system/fs-019a6b4db8268f5f0',
   'CreationTime': datetime.datetime(2022, 6, 29, 20, 26, 59, tzinfo=tzlocal()),
   'LifeCycleState': 'available',
   'Name': 'dataset_xbtusd-eu-west-2c',
   'NumberOfMountTargets': 1,
   'SizeInBytes': {'Value': 6144, 'ValueInIA': 0, 'ValueInStandard': 6144},
   'PerformanceMode': 'generalPurpose',
   'Encrypted': False,
   'ThroughputMode': 'bursting',
   'AvailabilityZoneName': 'eu-west-2c',
   'AvailabilityZoneId': 'eu

In [4]:
conn.describe_mount_targets(FileSystemId='fs-019a6b4db8268f5f0')

{'ResponseMetadata': {'RequestId': 'c4171f2c-efb5-4e04-96f0-dd7b462d4d2b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c4171f2c-efb5-4e04-96f0-dd7b462d4d2b',
   'content-type': 'application/json',
   'content-length': '394',
   'date': 'Wed, 29 Jun 2022 21:27:30 GMT'},
  'RetryAttempts': 0},
 'MountTargets': [{'OwnerId': '050980610391',
   'MountTargetId': 'fsmt-0357ad80676b438ae',
   'FileSystemId': 'fs-019a6b4db8268f5f0',
   'SubnetId': 'subnet-0a246fb0d80f581d4',
   'LifeCycleState': 'available',
   'IpAddress': '172.31.9.10',
   'NetworkInterfaceId': 'eni-0fd8abe253a7bf36f',
   'AvailabilityZoneId': 'euw2-az1',
   'AvailabilityZoneName': 'eu-west-2c',
   'VpcId': 'vpc-03e1442d2baab989f'}]}

In [9]:
%%sh 

sudo mount -t efs -o tls fs-019a6b4db8268f5f0:/ efs
    
sudo chmod go+rw ./efs

Failed to resolve "fs-019a6b4db8268f5f0.efs.eu-west-2.amazonaws.com". Cannot connect to file system mount target ip address 172.31.9.10. 
Connection to the mount target IP address 172.31.9.10 timeout. Please retry in 5 minutes if the mount target is newly created. Otherwise check your VPC and security group configuration to ensure your file system is reachable via TCP port 2049 from your instance.


In [63]:
# https://public.bitmex.com/?prefix=data/trade/
endpoint = 'https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/trade/{}.csv.gz'

In [46]:
def scrape(year, date, end):
    end_date = min(dt(year, 12, 31), dt.today() - timedelta(days=1))

    while date <= end_date and date <= end:
        date_str = date.strftime('%Y%m%d')
        print("Processing {}...".format(date))
        count = 0
        while True:
            r = requests.get(endpoint.format(date_str))
            if r.status_code == 200:
                break
            else:
                count += 1
                if count == 10:
                    r.raise_for_status()
                print("Error processing {} - {}, trying again".format(date, r.status_code))
                time.sleep(10)
        folder = "efs/"
        with open(folder + date_str, 'wb') as fp:
            fp.write(r.content)

        with gzip.open(folder + date_str, 'rb') as fp:
            data = fp.read()

        with open(folder + date_str + 'temp', 'wb') as fp:
            fp.write(data)

        with open(folder + date_str + 'temp', 'r') as file_in:
            with open(folder + date_str, "w") as file_out:
                file_out.writelines(filter(lambda line: ',XBTUSD,' in line, file_in))

        os.remove(folder + date_str + 'temp')
        date += timedelta(days=1)


def merge(year):
    print("Generating CSV for {}".format(year))
    files = sorted(glob.glob("{}*".format(year)))
    first = True
    with open("{}.csv".format(year), 'wb') as out:
        for f in files:
            with open(f, 'rb') as fp:
                if first is False:
                    fp.readline()
                first = False
                shutil.copyfileobj(fp, out)
    for f in files:
        os.unlink(f)

In [69]:
#parser = argparse.ArgumentParser(description='BitMex historical data scraper. Scrapes files into single year CSVs')
#parser.add_argument('--start', default="20190101", help='start date, in YYYYMMDD format. Default is 2014-11-22, the earliest data date for BitMex')
#parser.add_argument('--end', default=None, help='end date, in YYYYMMDD format. Default is yesterday')
#args = parser.parse_args()

#start = dt.strptime(args.start, '%Y%m%d')
#end = dt.strptime(args.end, '%Y%m%d') if args.end else dt.utcnow()

start = dt.strptime("20190101", '%Y%m%d')
end = dt.utcnow()

years = list(range(start.year, end.year + 1))

starts = [dt(year, 1, 1) for year in years]
starts[0] = start

for year, start in zip(years, starts):
    scrape(year, start, end)
    merge(year)

Processing 2019-01-01 00:00:00...
Processing 2019-01-02 00:00:00...
Processing 2019-01-03 00:00:00...
Processing 2019-01-04 00:00:00...
Processing 2019-01-05 00:00:00...
Processing 2019-01-06 00:00:00...


KeyboardInterrupt: 

In [8]:
%tb

SystemExit: 2